In [6]:
import xarray as xr
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
write_path = '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/'
read_path = '/uio/lagringshotell/geofag/students/metos/hannasv/dataERA5_grib/'

base = '/uio/lagringshotell/geofag/students/metos/hannasv/'
base = '/home/hanna/lagrings'
#read_path = '/uio/lagringshotell/geofag/students/metos/hannasv/dataERA5_grib/'
read_folder = 'ERA5_monthly'

In [7]:
folder = os.path.join(base, read_folder)

In [8]:
folder

'/home/hanna/lagrings/ERA5_monthly'

In [10]:
files = glob.glob(os.path.join(folder, '*.nc'))

895

In [218]:
# files = glob.glob(os.path.join(read_path, '*.grib'))

In [30]:
# domain

<xarray.Dataset>
Dimensions:        (latitude: 81, longitude: 161, time: 61368)
Coordinates:
    number         int64 ...
  * time           (time) datetime64[ns] 2012-01-01 ... 2018-12-31T23:00:00
    step           timedelta64[ns] ...
    isobaricInhPa  int64 ...
  * latitude       (latitude) float64 50.0 49.75 49.5 49.25 ... 30.5 30.25 30.0
  * longitude      (longitude) float64 -15.0 -14.75 -14.5 ... 24.5 24.75 25.0
    valid_time     (time) datetime64[ns] ...
Data variables:
    q              (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2020-02-20T09:59:12 GRIB to CDM+CF via cfgrib-0....

In [ ]:
data = xr.open_dataset(files[6], engine = 'cfgrib')
domain = data.sel(latitude = slice(50,30), longitude = slice(-15, 25))

In [ ]:
def dataset_to_numpy(pixel, bias = True):
    """ Dataset. """ 
    n = len(pixel.time.values)
    
    if bias:
        num_vars = 4+1
    X = np.zeros((n, num_vars))
    
    q   = pixel.q.values
    t2m   = pixel.t2m.values
    r   = pixel.r.values
    sp  = pixel.sp.values
    tcc = pixel.tcc.values
        
    X[:, 0] = q
    X[:, 1] = t2m
    X[:, 2] = r
    X[:, 3] = sp
    X[4, :] = 1
    return X, tcc[:, np.newaxis]

In [98]:
domain

<xarray.Dataset>
Dimensions:        (latitude: 81, longitude: 161, time: 70128)
Coordinates:
    number         int64 ...
  * time           (time) datetime64[ns] 2004-01-01 ... 2011-12-31T23:00:00
    step           timedelta64[ns] ...
    isobaricInhPa  int64 ...
  * latitude       (latitude) float64 50.0 49.75 49.5 49.25 ... 30.5 30.25 30.0
  * longitude      (longitude) float64 -15.0 -14.75 -14.5 ... 24.5 24.75 25.0
    valid_time     (time) datetime64[ns] ...
Data variables:
    r              (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2020-02-20T19:31:32 GRIB to CDM+CF via cfgrib-0....

In [99]:
var = 'r'
months = np.arange(1, 13)
#years = np.arange(2012, 2019)
years=np.arange(2004, 2012)

In [106]:
for m in months:
    for y in years:
        subset = domain.sel(time = '{}-{:02d}'.format(y, m))
        filename = '{}_{:02d}_{}.nc'.format(y, m, var)
        eng = 'netcdf4'
        encoding_dict = {'{}'.format(var): {'zlib': True, 'complevel': 9}}
        subset.to_netcdf(os.path.join(write_path, filename), encoding=encoding_dict, engine = eng)

# Deal with missing values
## Merge datasets and only keep the times present in both 

In [23]:
import xarray as xr
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import scipy 

write_path = '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/'
read_path = '/uio/lagringshotell/geofag/students/metos/hannasv/dataERA5_grib/'

In [43]:
results_path = '/uio/lagringshotell/geofag/students/metos/hannasv/results/ar/models/'

['/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/2012_05_q.nc',
 '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/2012_05_r.nc',
 '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/2012_05_t2m.nc',
 '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/2012_05_sp.nc',
 '/uio/lagringshotell/geofag/students/metos/hannasv/ERA5_monthly/2012_05_tcc.nc']

In [19]:
def dataset_to_numpy(pixel, bias = True):
    """ Dataset. """ 
    n = len(pixel.time.values)
    
    if bias:
        num_vars = 4+1
    X = np.zeros((n, num_vars))
    
    q   = pixel.q.values
    t2m   = pixel.t2m.values
    r   = pixel.r.values
    sp  = pixel.sp.values
    tcc = pixel.tcc.values
    
    
    X[:, 0] = q
    X[:, 1] = t2m
    X[:, 2] = r
    X[:, 3] = sp
    X[4, :] = 1
    return X, tcc[:, np.newaxis]

In [20]:
# Store R2, MSE, Weights, 
# Names Wt2m, Wr, Wq, Wsp, W1, W2, .... Wn

In [28]:
from scipy.linalg import inv

In [42]:
files = glob.glob(os.path.join(write_path, '2012_05_*.nc'))
data = [xr.open_dataset(fil) for fil in files]
most = xr.merge(data, 'equals')
res = np.zeros((n_lat, n_lon, len(coeffs)))
latitudes = most.latitude.values
longitudes = most.longitude.values
    
n_lat = 81
n_lon = 161

for i, lat in enumerate(latitudes): # 81
    for j, lon in enumerate(longitudes): # 161

        pixel = most.sel(latitude = lat, longitude = lon)
        X, y = dataset_to_numpy(pixel)
        coeffs = np.dot(inv(np.dot(X.T, X)), np.dot(X.T, y))
        res[i, j, :] =  coeffs.flatten()

In [45]:
temp_dict = {}
order = 3

for i in range(order):
    var = 'W{}'.format(i+1)
    temp_dict[var] = (['latitude', 'longitude'], np.zeros((n_lat, n_lon)))
    
vars_dict = {'b': (['latitude', 'longitude'], res[:, :, 4]),
             'Wt2m':(['latitude', 'longitude'], res[:, :, 1]),
             'Wr': (['latitude', 'longitude'], res[:, :, 2]),
             'Wq':(['latitude', 'longitude'], res[:, :, 0]),
             'Wsp': (['latitude', 'longitude'], res[:, :, 3]),
              }
temp_dict.update(vars_dict)
ds = xr.Dataset(temp_dict,
                 coords={'longitude': (['longitude'], longitudes),
                         'latitude': (['latitude'], latitudes),
                        })

In [46]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161)
Coordinates:
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
Data variables:
    b          (latitude, longitude) float64 -44.97 -49.1 -104.5 ... 56.62 51.73
    Wq         (latitude, longitude) float64 45.4 49.76 105.4 ... -55.65 -50.77
    Wt2m       (latitude, longitude) float64 -0.1195 -0.1336 ... 0.02458 0.02495
    Wsp        (latitude, longitude) float64 0.0003534 0.0003937 ... -6.835e-05
    W3         (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    W2         (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    W1         (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    Wr         (latitude, longitude) float64 -0.01508 -0.0156 ... 0.01022

# Store the results from the regression in nc-files

* 0: q
* 1: t2m
* 2: r
* 3: sp
* 4: bias

In [47]:
# Inplace returns None
encoding_dict = {}    
for key in temp_dict.keys():
    encoding_dict[key] = {'zlib': True, 'complevel': 9}

In [50]:
ds.to_netcdf(path = os.path.join(results_path,  'test.nc'),
             engine='netcdf4',
             encoding =encoding_dict)

In [44]:
# include training periode in attributes and filename

## Read results file and make a prediction.

In [51]:
results = xr.open_dataset(os.path.join(results_path,  'test.nc'))

In [58]:
def predict(X, coeffs):
    """Make prediction"""
    return np.dot(X.T, coeffs)

##  Make prediction and calculate R2 and MSE both training and test
### This file contains
* MSE_train
* MSE_test

* R2_train
* R2_test

In [52]:
results

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161)
Coordinates:
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
Data variables:
    b          (latitude, longitude) float64 ...
    Wq         (latitude, longitude) float64 ...
    Wt2m       (latitude, longitude) float64 ...
    Wsp        (latitude, longitude) float64 ...
    W3         (latitude, longitude) float64 ...
    W2         (latitude, longitude) float64 ...
    W1         (latitude, longitude) float64 ...
    Wr         (latitude, longitude) float64 ...

In [59]:
files = glob.glob(os.path.join(write_path, '2012_06_*.nc'))
data = [xr.open_dataset(fil) for fil in files]
most = xr.merge(data, 'equals')

In [216]:
most

<xarray.Dataset>
Dimensions:        (latitude: 81, longitude: 161, time: 720)
Coordinates:
  * latitude       (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * time           (time) datetime64[ns] 2012-06-01 ... 2012-06-30T23:00:00
    number         int64 0
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 1000
  * longitude      (longitude) float64 -15.0 -14.75 -14.5 ... 24.5 24.75 25.0
    valid_time     (time) datetime64[ns] 2012-06-01 ... 2012-06-30T23:00:00
    surface        int64 0
Data variables:
    q              (time, latitude, longitude) float32 0.010645431 ... 0.010000957
    r              (time, latitude, longitude) float32 79.789215 ... 73.236374
    t2m            (time, latitude, longitude) float32 293.0703 ... 290.511
    sp             (time, latitude, longitude) float32 101912.125 ... 98598.375
    tcc            (time, latitude, longitude) float64 0.0 0.0 ... 1.0 1.0
    nr_nans        (time, latitude, longitude) float64 0.0 0.0

In [217]:
81*161*720

9389520

In [219]:
import seaborn as sns

In [221]:
temp = most.tcc.values.mean(axis=0)

In [222]:
temp.shape

(81, 161)

In [226]:
#temp.nanmean()

In [223]:
sns.heatmap(temp)

ValueError: zero-size array to reduction operation minimum which has no identity

In [57]:
variable = results.variables.keys()

## Compare results to normalized predictions.

In [ ]:
def prep_data_for_prediction(data, order):
    """ Preperation of data to make prediction
    
    data : xr.Dataset 
    
    order : int
        The n-th order of autoregressive model. Number of past cloud cover included.     
    """
    
    # Determine how many times have information about for cloud cover in 'order' previous steps.
    X = None
    
    return X

In [61]:
def results_to_numpy(data, weights):
    """ Dataset. 
    Use weights to determine the order of the AR model.
    
    
    data : xr.Dataset
        contains the data you want to make a prediction on 
    
    weights : xr.Dataset
        Contains the weight and bias used to train this model.
        
    """ 
    weights_keys = weights.variables.keys()
    
    order = len(weights_keys) - 5
    
    # Call function which prepares the data to a format suitable to make a prediction. 
    # This involves 
    test_data = prep_data_for_prediction(data, order)
    weights = pred_weights(weights)
    
    
    
    n = len(dataset.time.values)
    
    if bias:
        num_vars = 4+1
        
    X = np.zeros((n, num_vars))
    
    q   = dataset.q.values
    t2m = dataset.t2m.values
    r   = dataset.r.values
    sp  = dataset.sp.values
    tcc = dataset.tcc.values
    
    X[:, 0] = q
    X[:, 1] = t2m
    X[:, 2] = r
    X[:, 3] = sp
    X[:, 4] = 1
    return X, tcc[:, np.newaxis]

In [63]:
results

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161)
Coordinates:
  * latitude   (latitude) float64 30.0 30.25 30.5 30.75 ... 49.5 49.75 50.0
  * longitude  (longitude) float64 -15.0 -14.75 -14.5 -14.25 ... 24.5 24.75 25.0
Data variables:
    b          (latitude, longitude) float64 ...
    Wq         (latitude, longitude) float64 ...
    Wt2m       (latitude, longitude) float64 ...
    Wsp        (latitude, longitude) float64 ...
    W3         (latitude, longitude) float64 ...
    W2         (latitude, longitude) float64 ...
    W1         (latitude, longitude) float64 ...
    Wr         (latitude, longitude) float64 ...

In [71]:
variables

[u'b',
 u'Wq',
 u'Wt2m',
 u'Wsp',
 u'W3',
 u'W2',
 u'W1',
 u'Wr',
 u'latitude',
 u'longitude']

In [172]:
def get_weights(dataset):
    """
    weights: xr.Dataset
        contains the weights and biases
    
    Returns
    --------------
    
    X : weights in matrix for 
        
    explenation : List[str]
        Exlpaines the content in dimension 2.
    
    TODO: 
        Make sure they come in the correct oder
        
    """
    variables = dataset.variables.keys()    
    X = np.zeros((n_lat, n_lon, len(variables)))
    
    W_q   = dataset.Wq.values
    W_t2m = dataset.Wt2m.values
    W_r   = dataset.Wr.values
    W_sp  = dataset.Wsp.values
    b = dataset.b.values
    
    X[:, :, 0] = W_q[:,:]
    X[:, :, 1] = W_t2m[:,:]
    X[:, :, 2] = W_r[:,:]
    X[:, :, 3] = W_sp[:,:]
    X[:, :, 4] = b # bias
    
    if len(variables) > 5+2: # 2 since it contains the latitude, longitude information.
        for i in range(1, len(variables)-2-5+1):
            var = 'W{}'.format(i)
            X[:, :, i+4] = dataset[var].values                      
    return X

In [173]:
#results.Wq

# Create boolean arrays for detecting if one value has values back in time. 
# Create copies of the time axis and check if there is an hour between then 

In [203]:
def get_predict_data(dataset, order):
    """
    dataset : xr.Dataset
        Contains the data you want to make a prediction based.
    
    TODO add explination column???
        
    keep the order of xarray time, lat, lon
    """
    times = dataset.time.values
    X = np.zeros( (len(times)-order, n_lat, n_lon, order+5) )
    y = np.zeros( (len(times)-order, n_lat, n_lon) )
    # X.shape = (lat, lon, variables, times) 
    
    q   = dataset.q.values
    t2m = dataset.t2m.values
    r   = dataset.r.values
    sp  = dataset.sp.values
    tcc = dataset.tcc.values
    
    X[:, :, :, 0] = q[:-order,:,:]
    X[:, :, :, 1] = t2m[:-order,:,:]
    X[:, :, :, 2] = r[:-order,:,:]
    X[:, :, :, 3] = sp[:-order,:,:]
    X[:, :, :, 4] = 1 # bias
    
    y[:, :, :] = tcc[:-order]
    
    index = 5
    
    # tcc1, tcc2, ..., tcc_n
    for temp_order in range(1, order+1):
        a = times[:-temp_order]
        b = times[temp_order:]
        bo = [element.astype(int) == temp_order for element in (b-a).astype('timedelta64[h]') ]
        
        remove_from_end = order - temp_order
        if remove_from_end != 0:
            #remove_from_end = 1
            # Which clouds to add at which column, remember that they shoudl start from t-1, t-2, t-3 ...  
            X[:, :, :, index] = tcc[temp_order:, :, :][bo][:-remove_from_end, :, :]
        else:
            X[:, :, :, index] = tcc[temp_order:, :, :][bo]
        index+=1     
    return X, y

In [204]:
data, true = get_predict_data(most, order=2)

In [178]:
X.shape

(81, 161, 10)

In [227]:
X = get_weights(results)

In [228]:
np.isnan(X).sum()

0

In [179]:
X = get_weights(results)[:, :, :7]

In [180]:
X.shape

(81, 161, 7)

In [182]:
data.shape

(718, 81, 161, 7)

In [185]:
test = data[0, :, :, :]*X

In [186]:
test.shape

(81, 161, 7)

In [188]:
prediction_t0 = test.sum(axis=2)

In [200]:
prediction = [(data[i, :, :, :]*X).sum(axis=2) for i in range(718)]

In [201]:
np.array(prediction).shape

(718, 81, 161)

# Make predictions 

In [208]:
def mean_squared_error(y_true, y_pred):
    """Computes the Mean Squared Error score metric."""
    mse = np.square(np.subtract(y_true, y_pred)).mean(axis = 0)
    return mse

def r2_score(y_true, y_pred):
    """  """
    numerator = np.square(np.subtract(y_true, y_pred)).sum()
    denominator = np.square(np.subtract(y_true, np.average(y_true))).sum()
    val = numerator/denominator
    return 1 - val

In [ ]:
def generate_true_values_to_compare_against():
    # TODO tomorrow 

In [205]:
true.shape

(718, 81, 161)

In [212]:
mse = mean_squared_error(true, prediction)

In [214]:
true.mean(axis = 0)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [213]:
mse

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])